## XLNET for sequence classification

In [1]:
# Let's compare BERT with XLNET
from transformers import BertTokenizer, BertModel
  
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

bert_model = BertModel.from_pretrained("bert-base-cased")


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# Look for gpu to use. Will use 'cpu' by default if no gpu found.
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
local_path = "../"
remote_path = "/notebooks/oreilly-hands-on-transformers/"

#runtime_path = remote_path
runtime_path = local_path

In [4]:
one_encoded = bert_tokenizer.encode_plus('How much will this cost?', add_special_tokens=True, return_tensors='pt')
two_encoded = bert_tokenizer.encode_plus('Is it expensive?', add_special_tokens=True, return_tensors='pt')


In [5]:
one_encoded

{'input_ids': tensor([[ 101, 1731, 1277, 1209, 1142, 2616,  136,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
two_encoded

{'input_ids': tensor([[ 101, 2181, 1122, 5865,  136,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [7]:
# the CLS token is at the beginning in BERT
one_embedded = bert_model(**one_encoded).last_hidden_state[:,0,:]
two_embedded = bert_model(**two_encoded).last_hidden_state[:,0,:]


In [8]:
import torch

torch.nn.CosineSimilarity()(one_embedded, two_embedded)

tensor([0.9723], grad_fn=<SumBackward1>)

In [9]:
from transformers import XLNetTokenizer, XLNetModel
  
xlnet_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

xlnet_model = XLNetModel.from_pretrained("xlnet-base-cased")


Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
xlnet_model

XLNetModel(
  (word_embedding): Embedding(32000, 768)
  (layer): ModuleList(
    (0): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, bias=True)
        (layer_2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (activation_function): GELUActivation()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_

In [11]:
one_encoded = xlnet_tokenizer.encode_plus('How much will this cost?', add_special_tokens=True, return_tensors='pt')
two_encoded = xlnet_tokenizer.encode_plus('Is it expensive?', add_special_tokens=True, return_tensors='pt')

In [12]:
xlnet_tokenizer.convert_ids_to_tokens(one_encoded['input_ids'][0])

['▁How', '▁much', '▁will', '▁this', '▁cost', '?', '<sep>', '<cls>']

In [13]:
# the CLS token is at the end in XLNET
one_embedded = xlnet_model(**one_encoded).last_hidden_state[:,-1,:]
two_embedded = xlnet_model(**two_encoded).last_hidden_state[:,-1,:]


In [14]:
torch.nn.CosineSimilarity()(one_embedded, two_embedded)

tensor([0.9734], grad_fn=<SumBackward1>)

## Fine-tuning XLNET

In [15]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from datasets import Dataset

In [16]:
# Ingest 100 tweets from the Kaggle disaster tweet comopetition
import pandas as pd

tweets = pd.read_csv(runtime_path + 'data/disaster_sample.csv')

tweets.head(2)

,index,id,keyword,location,text,target,label
0,7138,10224,volcano,NaN,@MrMikeEaton @Muazimus_Prime hill hill mountai...,1,1
1,2151,3086,deaths,Blackpool,Cancers equate for around 25% of all deaths in...,1,1


In [17]:
tweet_dataset = Dataset.from_pandas(tweets)

# We will pad our dataset so that our input matrices are the same length and truncate anything longer than 512 tokens
def preprocess(data):
    return xlnet_tokenizer(data['text'], padding=True, truncation=True)
# In general I recommend using a collator with padding, but this is how it would be done
#  without a collator

tweet_dataset = tweet_dataset.map(preprocess, batched=True, batch_size=len(tweet_dataset))

# Dataset has a built in train test split method

tweet_dataset = tweet_dataset.train_test_split(test_size=0.2)

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
from transformers import AutoModelForSequenceClassification

In [19]:
xlnet_sequence_classification_model = XLNetForSequenceClassification.from_pretrained(
    'xlnet-base-cased', num_labels=2
)


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [21]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

training_args = TrainingArguments(
    output_dir= runtime_path + 'xlnet_clf',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps=len(tweet_dataset['train']) // 5,  # number of warmup steps for learning rate scheduler,
    weight_decay = 0.05,
    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

# Define accuracy metric:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define the trainer:

trainer = Trainer(
    model=xlnet_sequence_classification_model,
    args=training_args,
    train_dataset=tweet_dataset['train'],
    eval_dataset=tweet_dataset['test'],
    compute_metrics=compute_metrics
)

# Get initial metrics
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: keyword, target, location, text, index, id. If keyword, target, location, text, index, id are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 32


{'eval_loss': 0.7694569826126099,
 'eval_accuracy': 0.35,
 'eval_runtime': 10.1931,
 'eval_samples_per_second': 3.924,
 'eval_steps_per_second': 0.196}

In [22]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: keyword, target, location, text, index, id. If keyword, target, location, text, index, id are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
/home/apulache/Workspace/mambaforge/envs/Transformers/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 160
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 50
  Number of trainable parameters = 117310466


Epoch,Training Loss,Validation Loss,Accuracy
1,0.747500,0.716396,0.525000
2,0.652900,0.639310,0.700000
3,0.656000,0.609965,0.675000
4,0.506000,0.558303,0.675000
5,0.451800,0.592810,0.775000
6,0.310600,0.558206,0.800000
7,0.176300,0.622536,0.800000
8,0.138300,0.755227,0.800000
9,0.161200,0.730602,0.825000
10,0.013000,0.783955,0.825000


The following columns in the evaluation set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: keyword, target, location, text, index, id. If keyword, target, location, text, index, id are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 32
Saving model checkpoint to ../xlnet_clf/checkpoint-5
Configuration saved in ../xlnet_clf/checkpoint-5/config.json
Model weights saved in ../xlnet_clf/checkpoint-5/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: keyword, target, location, text, index, id. If keyword, target, location, text, index, id are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 32
Saving mo

TrainOutput(global_step=50, training_loss=0.4064293984882534, metrics={'train_runtime': 1493.6833, 'train_samples_per_second': 1.071, 'train_steps_per_second': 0.033, 'total_flos': 70329819014400.0, 'train_loss': 0.4064293984882534, 'epoch': 10.0})

In [23]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: keyword, target, location, text, index, id. If keyword, target, location, text, index, id are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 32


{'eval_loss': 0.5582063794136047,
 'eval_accuracy': 0.8,
 'eval_runtime': 10.6341,
 'eval_samples_per_second': 3.761,
 'eval_steps_per_second': 0.188,
 'epoch': 10.0}